# **Traffic Incidents optimization**

## **Context**

In some high-density traffic cities like Bogotá, slow attention to traffic incidents remains an issue, and a method to deal with it is yet to be implemented. Quick response is important for most drivers. Those involved in accidents would have higher chances of survival, and even those never involved in an incident would see the benefit of faster moving traffic.

Let us define what we mean by traffic incident or accident. Firstly, an accident implies the happening of a crash, which could be one of a car against an object, or between vehicles. On the other hand, incident is a more general term, it refers to any event that negatively affects traffic flow, so in those terms, an accident is a specific kind of incident.

In 2019, Bogotá had the alarming rate of a traffic accident every five to six minutes, and in 2018 there were a total of 50 800 traffic incidents, a number excesively high, that if it were to be reduced, a great quantity of time would be saved. This of course puts interest on projects trying to fix this problem, which can take many different approaches and focal points. Among these are 



## **Objectives**

## **Model formulation**

El modelo para la solución a nuestro problema se divide en 2 partes, la primera parte  se encarga de  maximizar los pesos totales  de la gravedad de los incidentets que se abordarán con los recursos de respuesta actualmente  disponible, y la  segunda parte se encarga de minimizar el tiempo tatal de viaje de la respuesta de todas las unidades.

$\textbf{Fase 1:}$
La función objetivo en esta fase es: 

$(1a)$ $$\max \sum_{j=1}^{N_i}a_{s(j)}z_j$$

bajo las reestricciones:

$(1b)$ $$\sum_{i=1}^{N_t} x_{ij}\geq b_{s(j)} z_j,\hspace{1cm} \forall j\in N_i$$ 

$(1c)$  $$\sum_{i=1}^{N_i} x_{ij}\leq 1,\hspace{1cm} \forall i\in RU $$ 

$(1d)$  $$x_{ij}=0 \text{ o } 1, \hspace{1cm} \forall i\in RU, \forall j\in N_i$$

$(1e)$   $$z_{j}=0 \text{ o } 1, \hspace{1cm}, \forall j\in N_i$$

$\textbf{Fase 2: }$ La función objetivo en esta parte es:

$(2a)$ $$\min \sum_{i=1}^{N_t}\sum_{j=1}^{N_i} x_{ij}t_{ij}(t)$$

bajo las restricciones: 

$(2b)$ $$\sum_{i=1}^{N_t} x_{ij}\geq b_{s(j)} z_j,\hspace{1cm} \forall j\in N_i$$ 

$(2c)$  $$\sum_{i=1}^{N_i} x_{ij}\leq 1,\hspace{1cm} \forall i\in RU $$ 

$(2d)$  $$x_{ij}=0 \text{ o } 1, \hspace{1cm} \forall i\in RU, \forall j\in N_i$$


Donde 

$t_{ij}(t): $ tiempo de viaje del camino más corte entre los puntos $i$, $j$ en el paso de tiempo $t$

$S(j): $ La función que devuelve la prioridad de un incidente dado $j$ (este es un número entre 1 y 4, donde 1 es la menor prioridad)

$a_l: $ El peso de la gravedad de los incidentes con l-ésima prioridad

$b_l:$ El número de gruas necesarias para ser despachadas a un incidente con l-ésima prioridad(se necesitan 2 para una prioridad de 4, 1 para una prioridad de 3, 1 para una prioridad de 2, y 0 para una prioridad de 1)

$N_i:$ El número de incidentes que están esperando grúas 

$N_t:$ El número de gruas en todo en sistema

$RU: $ Unidades de respuesta 


Variables de desición

$z_j=1$  Si el incidente j-ésimo será tratado por ciertas grúas, de lo contrario 0

$x_{ij}=1$ Si la grúa i se despacha al incidente j, de lo contrio 0 

En la fase 1 del modelo, la función objetivo (1a) es máximizar los pesos totales de la gravedad de los incidentes a tratar
sin exceder la limitación de número de unidades de respuesta  disponibles. La restricción (1b) garantiza que se envíe una cantidad suficiente
de grúas a cada lugar del incidente. La restricción (1c) indica que una grúa solo puede atender un solo incidente en un
momento dado. Las restricciones (1d) y (1e) representan restricciones de enteros, la ecuacion (2a) se encarga de minimazar el tipo de viaje estimado de las unodades que se requieren para moverse desde sus ubicaciones acutales a las ubicaciones del incidente asigandas.

Este problema pertenece a la programación lineal entera (puesto que la función objetivo es lineal, tanto en la fase 1 como en la fase 2 es un producto punto entre 2 vectores, y las restricciónes son lineales y las variables de desción son entera), dado que este problema en el fondo es lineal tiene solución por medio de algoritmos que involucren convexidad, pero por las resctricciones enteras resulta que este problema no siempre va a tener solución de hecho es un problema NP-completo.

## **Model implementation**

Imports

In [1]:
import cvxpy as cp
import numpy as np
import pandas as pd
# np.set_printoptions(precision=3, suppress=True)

First step is to maximize the severities of the incidents under current response resources(Number of available agents)

In [2]:
i = 3 # number of agents
j = 5  # number of incidents

b = np.array([1, 2, 3, 2, 1]) # agents needed per incidents

In [3]:
print("Agents needed:\n", b, end="\n")

Agents needed:
 [1 2 3 2 1]


In [4]:
x = cp.Variable((i, j), boolean=True) # agents-incidents matrix
z = cp.Variable(j, boolean=True) # incidents coverage vector

objective = cp.Maximize(b@z) 

constraints = [cp.sum(x, axis=0) >= cp.multiply(b, z), # required agents per incidents
               cp.sum(x, axis=1) <= 1 # one agent covers at max. one incident
               ]
               
problem = cp.Problem(objective, constraints)

problem.solve(verbose=1)

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Jun 30 12:23:09 AM: Your problem has 20 variables, 2 constraints, and 0 parameters.
(CVXPY) Jun 30 12:23:09 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jun 30 12:23:09 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jun 30 12:23:09 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jun 30 12:23:09 AM: Compiling problem (target solver=SCIP).
(CVXPY) Jun 30 12:23:09 AM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> Cone

3.0

In [5]:
print("Agents :\n", x.value, end="\n\n")
print("Incidents:\n", z.value)

Agents :
 [[0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]

Incidents:
 [0. 0. 1. 0. 0.]


Next step is to minize the total travel time of all the agents

In [6]:
t = pd.read_csv("durations.csv")
t

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,381,382,383,384,385,386,387,388,389,390
0,0,1780.0,2203.2,1174.8,1206.3,1248.1,1156.2,2097.2,1963.0,1890.4,...,2385.5,956.1,2464.3,1151.2,1065.8,1749.0,1969.2,2574.2,1547.5,743.9
1,1,1041.6,1473.2,689.4,350.4,234.6,642.1,1367.2,1233.0,1160.4,...,1655.5,398.7,1734.3,637.1,551.7,1019.0,1253.6,1715.6,703.6,645.0
2,2,1166.2,1122.2,1389.0,1075.7,946.4,1367.4,1102.7,1118.3,1145.4,...,1317.0,950.8,1395.8,1362.4,1277.0,926.5,1110.2,1257.6,468.1,1483.4
3,3,1995.2,1342.2,2218.0,1893.6,1875.5,2185.3,1460.2,1693.6,1895.9,...,1674.5,1879.9,1711.4,2180.3,2094.9,1283.6,1731.0,1490.3,1312.1,2309.1
4,4,1356.2,1692.0,1062.7,839.7,422.7,1044.1,1631.1,1477.6,1423.4,...,1874.3,171.7,1953.1,1039.1,953.7,1282.9,1469.5,1885.3,722.1,662.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,987.5,1194.8,1210.3,901.1,770.3,1192.8,1095.8,939.6,966.7,...,1377.1,774.7,1455.9,1187.8,1102.4,759.0,931.5,1388.1,289.4,1307.3
96,96,473.3,1286.5,427.3,740.3,1066.3,586.3,940.9,864.8,799.4,...,1374.2,1090.0,1453.0,553.1,562.8,832.3,642.6,1657.5,833.5,798.0
97,97,1058.7,1394.5,953.9,614.9,411.7,906.6,1333.6,1199.2,1125.9,...,1576.8,416.1,1655.6,901.6,816.2,985.4,1177.3,1587.8,419.0,948.7
98,98,829.8,543.1,1238.2,1005.6,1323.2,1197.0,280.8,171.1,533.1,...,714.1,1346.9,792.9,1192.0,1106.6,417.3,382.9,914.1,896.1,1331.2


In [7]:
objective = cp.Minimize(cp.sum(cp.sum(cp.multiply(x, t), axis=0)))


constraints = [cp.sum(x, axis=0) >= cp.multiply(b, z.value), # required agents per incidents
               cp.sum(x, axis=1) <= 1 # one agent covers at max. one incident
               ]
               
problem = cp.Problem(objective, constraints)

problem.solve()

ValueError: Cannot broadcast dimensions  (3, 5) (100, 392)

In [ ]:
x.value

array([[0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]])

## **Model in practice**

In [ ]:
import pandas as pd

In [12]:
agents = pd.read_csv("agents_may.csv", parse_dates=["read_time", "time_stamp"])
incidents = pd.read_csv("incidents_may.csv", parse_dates=["incident_time"])

In [13]:
agents.head()

,id,dev_id,read_time,time_stamp,speed,grp_name,localidad,latitude,longitude,devicegroupid
0,1651381596,868033050089715,2022-05-01 00:06:36,2022-05-01 00:01:40,0.053087,S60,RAFAEL URIBE URIBE,4.551646,-74.108515,6
1,1651381596,868033050102534,2022-05-01 00:06:36,2022-05-01 00:01:42,0.000000,S60,KENNEDY,4.650030,-74.147322,6
2,1651381596,868033050090085,2022-05-01 00:06:36,2022-05-01 00:01:47,0.000000,S60,RAFAEL URIBE URIBE,4.581688,-74.128348,6
3,1651381596,868033050103508,2022-05-01 00:06:36,2022-05-01 00:01:52,0.000000,S60,CIUDAD BOLIVAR,4.551098,-74.147635,6
4,1651381596,868033050101650,2022-05-01 00:06:36,2022-05-01 00:01:55,0.000000,S60,KENNEDY,4.619840,-74.159668,6
